In [13]:
using BenchmarkTools, SpecialFunctions
using LinearAlgebra, Statistics
using Plots
using Random, Distributions
gr(fmt=:png); # setting for easier display in jupyter notebooks

**Introduction**

I'm teaching myself ``julia``. The simplest way for me todo so is to work with a model that I know extremely well---the Ricardian model of Eaton and Kortum (2002). So below I will outline the economic environment and then use the properties of ``julia`` to map the fundemental, mathematical enviorment into a computational/quantitative outcome. Outside of performance properties, the more I reflect on this, this is a huge appeal of the ``julia`` language. In contrast, in for example MATLAB, the map from white board to computation is big.

---

**Countries.** To keep things simple, let's firs work with a simple two country set up---a home and a foreign country. 

**Individual Goods Production.**
Within a country, there is a continuum of intermediate goods indexed by $\omega \in [0, 1]$. As in the Ricardian model of \citet{dornbusch1977comparative} and \citet{eaton2002technology}, intermediate goods are not nationally differentiated. Thus, intermediate $\omega$ produced in one country is a perfect substitute for the same intermediate $\omega$ produced by another country. Competitive firms produce intermediate goods with linear production technologies,
\begin{align}
q(\omega) = z(\omega) \ell,
\end{align}
where $z$ is the productivity level of firms and $\ell$ is the number of efficiency units of labor. 

**The Aggregator** In this model, intermediate goods are aggregated by a competitive final-goods producer. In Julia, what would be interesting is to make the aggregator generic and let the user specify how this works. But for know, we will follow the canonical situation with  who has a standard CES production function:

\begin{align}
Q = \left[ \int_0^1 q(\omega)^{\frac{\sigma - 1}{\sigma}} d\omega \right]^{\frac{\sigma}{\sigma - 1}},
\label{eq:ces}
\end{align}

where $q(\omega)$ is the quantity of individual intermediate goods $\omega$ demanded by the final-goods firm, and $\sigma$ is elasticity of substitution across variety. I want to talk through some properties/definitions related to this aggregator.

- The aggregator in (\ref{eq:ces}) gives rise to the following the demand curve (in quantities) for an individual variety:

\begin{align}
q(\omega) & = \left(\frac{p_h(\omega)}{P_h}\right)^{-\sigma}Q.
\label{eq:demand_curve}
\end{align}

where $Q$ is the aggregate demand for the final good; $P_h$ is the CES price index, defined below; And $p_h(\omega)$ is the price paid by the home country for good $\omega$ (which is not the same necessarily as the price offered by a home country supplier.

- The demand curve can also be expressed in shares, not quantities. So

\begin{align}
\frac{p_h(\omega)q(\omega)}{P_h Q} & = \left(\frac{p_h(\omega)}{P_h}\right)^{1-\sigma}.
\label{eq:demand_curve_share}
\end{align}

which says that the value of of good $\omega$ relative to total expenditures $P_h Q$ is given by the formula above. This helps as we can compute expenditure patterns while being agnostic about the actual size of the economy, i.e. $Q$. One useful relationship that builds on this is to condition on the source and aggregate to arrive at the share of goods purchased from another country. How does this work? 

\begin{align}
\pi_{h,f} = \int_{p_h = p_{h,f}} \left(\frac{p_h(\omega)}{P_h}\right)^{1-\sigma} d\omega
\end{align}

Where the notation here $\pi_{h,f}$ is the share of goods that the home country sources from the foreign country. This is equal do the integral over all goods purchased from them, hence, the conditioning where $p_h = p_{h,f}$ and then the inside of the brackets is the same as above. Easy.

- The **CES Price Index** is given by

\begin{align}
P_h  = \left [ \int_{0}^{1} p_h(\omega)^{1-\sigma} d\omega \right]^{\frac{1}{1-\sigma}}
\label{eq:ces_price_index}
\end{align}

which takes all the prices at the micro-level and the is like a harmonic mean of these prices. Also note how one can substitute \ref{eq:ces_price_index} into the share formula to get an expression of expenditures shares purely in terms of micro-level prices.

In [5]:
function ces_shares(p::Matrix{Float64}, source_country::Matrix{CartesianIndex{2}}, σ::Float64)
    # this will take in (i) prices purchased at and
    # a cartesian index that can determine country supplier
    # then a parameter for the CES aggregator.
    # I'm toying around with explicity typing here. Any thoughts?
    # When presented this way. One can see how any arbitrary aggregator could be put in place.
    
    ngoods = length(p) # number of goods
    
    ncntry = ndims(source_country) # number of coutnries
    
    price_index = ( sum(p.^(1-σ)) / ngoods )^(1/(1-σ)) # CES price index
    
    # now construct the shares...
    trade_shares = zeros(ncntry)
    
    source_prob = zeros(ncntry)
    
    for cntry = 1:ncntry
        # This is the loop that conditions on the source country
        # first fix a country...

        supplier = [ zzz[2] == cntry for zzz in source_country ]
        # this is then a simple inline expression saying, for each 
        # index in the source country, which onex zzz[2] are equall 
        # to the country of origin...
        
        source_prob[cntry] = sum(supplier) / ngoods
        # This will compute the simple probability of being the low cost 
        # supplier. In the Frechet model = trade share. Alternative distributional
        # assumptions will deliver something different.
        
        trade_shares[cntry] = sum((p[supplier]./price_index).^(1-σ)) / ngoods
        
    end
        
    return price_index, trade_shares, source_prob

end

ces_shares (generic function with 1 method)

In [52]:
function marginal_cost(z::AbstractArray, wage::AbstractArray, τ::AbstractArray)
    return (τ.*wage)./z
end

marginal_cost (generic function with 2 methods)

In [47]:
function make_productity(T::Matrix{Int64},θ::Float64,ngoods::Int64)
    
    ncntry = length(T)
    
    z = zeros(ngoods,ncntry)
    
    for cntry = 1:ncntry
    
        dist = Frechet(θ, T[cntry]^(1/θ))
    
        z[:,cntry] = rand(dist, ngoods, 1);
    
    end
    
    return z
    
end

make_productity (generic function with 3 methods)

In [61]:
tau_matrix = [1 1.5; 1.5 1]
T = [1.0 2.0]
w = [1.0 1.0]
θ = 4.0

ncntry = ndims(tau_matrix)
ngoods = 1000000

z = make_productity(T,θ,ngoods)

price_index = zeros(ncntry)
shares = zeros(ncntry,ncntry)
source_prob = zeros(ncntry,ncntry)

for cntry = 1:ncntry

    p, source_country = findmin(marginal_cost(z, w, tau_matrix[cntry,:]'); dims = 2);
    # find the min over marginal costs
    # note that the tau_matrix returns a vector, not a row vector as I was anticipating
    # so take the transpose to make it work.

    (price_index[cntry], shares[cntry,:], source_prob[cntry,:]) = ces_shares(p, source_country, 2.0);
    # then this is set up so the row is the buyer...then you go across the columns
    # and that is share of goods the home country buys from that country.
    
end

In [62]:
shares

2×2 Matrix{Float64}:
 0.71748    0.28252
 0.0903151  0.909685

In [63]:
source_prob

2×2 Matrix{Float64}:
 0.717294  0.282706
 0.090292  0.909708

In [20]:
T = 2
d = Frechet(θ, T^(1/θ))
z = rand(d, 1000000, 1);

In [21]:
mean(z)

1.4567472473027496

In [22]:
T^(1/θ).*gamma(1-1/θ)

1.4572742614147616